##1.Naive Bayes Classifier

In [3]:
# Tập dữ liệu ví dụ

train_x = [
           'just plain boring',
           'entirely predictable and lacks energy',
           'no surprises and very few laughs',
           'very powerful',
           'the most fun film of the summer'
]
train_y = [0, 0, 0, 1, 1]

###1.1. Tiền xử lý dữ liệu cơ bản

In [4]:
def basic_preprocess(text):
    """ Tiền xử lý và tách các câu
    Args:
        text: câu đầu vào. 
        VD: "Tôi đi học"
    Output:
        text_clean: danh sách các từ (token) sau khi chuyển sang chữ thường và
            được phân tách bởi khoảng trắng
    """
    text_clean = text.lower()
    return text_clean.split()

basic_preprocess(train_x[0])

['just', 'plain', 'boring']

###1.2.Xây dựng bộ từ điển

In [5]:
# Ex 1
import tqdm
def count_freq_words(corpus, labels):
    """ Xây dựng bộ từ điển tần suất xuất hiện của các từ
    Args:
        corpus: tập danh sách các câu
        labels: tập nhãn tương ứng với các câu trong corpus (0 hoặc 1)
    Output:
        model: bộ từ điển ánh xạ mỗi từ và tần suất xuất hiện của từ đó trong corpus
            key: (word, label)
            value: frequency
            VD: {('boring', 0): 2} => từ boring xuất hiện 2 lần trong các sample thuộc class 0
    """
    model = {}
    for label, sentence in tqdm.tqdm(zip(labels, corpus)):
        for word in basic_preprocess(sentence):
            # Định nghĩa key của model là tuple (word, label)
            pair = (word, label)
            # Nếu key đã tồn tại trong model thì tăng value lên 1
            if pair in model:
                model[pair] += 1
            # Nếu key chưa tồn tại trong model thì bổ sung key vào model với value =1
            else:
                model[pair] = 1
    return model

In [6]:
#Kết quả mong đợi
freqs = count_freq_words(train_x, train_y)
freqs

5it [00:00, 4958.98it/s]


{('just', 0): 1,
 ('plain', 0): 1,
 ('boring', 0): 1,
 ('entirely', 0): 1,
 ('predictable', 0): 1,
 ('and', 0): 2,
 ('lacks', 0): 1,
 ('energy', 0): 1,
 ('no', 0): 1,
 ('surprises', 0): 1,
 ('very', 0): 1,
 ('few', 0): 1,
 ('laughs', 0): 1,
 ('very', 1): 1,
 ('powerful', 1): 1,
 ('the', 1): 2,
 ('most', 1): 1,
 ('fun', 1): 1,
 ('film', 1): 1,
 ('of', 1): 1,
 ('summer', 1): 1}

In [9]:
# Hàm lấy ra tần suất xuất hiện là value trong `freq` dựa vào key (word, label)
def lookup(freqs, word, label):
    '''
    Args:
        freqs: a dictionary with the frequency of each pair
        word: the word to look up
        label: the label corresponding to the word
    Output:
        count: the number of times the word with its corresponding label appears.
    '''
    count = 0

    pair = (word, label)
    if pair in freqs:
        count = freqs[pair]

    return count

lookup(freqs, "very", 0)

1

###1.3.Thuật toán phân loại Naive Bayes
**Bước 1: Tính xác suất tiên nghiệm của các class**
- Tính $D$, $D_{pos}$, $D_{neg}$
    - Dựa vào `train_y` tính số lượng các sample có trong tập training: $D$, số lượng các sample là positive (nhãn 1): $D_{pos}$ và số lượng nhãn là negative (nhãn 0): $D_{neg}$
    - Tính xác suất tiên nghiệm của class 1 là: $P(D_{pos})=D_{pos}/D$, và class 0 là: $P(D_{pos})=D_{pos}/D$

In [6]:
def compute_prior_prob(train_y):
    # Tính D, D_pos, D_neg dựa vào x_train
    ### START CODE HERE
    # Tính D, số lượng các sample trong training
    D = len(train_y)

    # Tính D_pos, số lượng các positive sample trong training
    D_pos = len(list(filter(lambda x: x == 1, train_y)))

    # Tính D_neg, số lượng các negative sample trong training
    D_neg = len(list(filter(lambda x: x == 0, train_y)))

    ### END CODE HERE
    # Tính xác suất tiên nghiệm cho các class 0 và 1
    p_prior = {0:(D_neg/D), 1:(D_pos/D)}
    return p_prior

In [7]:
# Kết quả mong đợi
compute_prior_prob(train_y)

{0: 0.6, 1: 0.4}

**Bước 2: Tính xác suất likelihood**
- Tính $V$: Dựa vào `freqs` tính số lượng các từ duy nhất (uniqe words) - gọi là bộ từ điển

- Tính $N_{pos}$ và $N_{neg}$: Dựa vào `freqs` dictionary, tính tổng số từ (có thể trùng lặp) xuất hiện trong positive samples $N_{pos}$ và negative samples $N_{neg}$.

- Tính tần suất xuất hiện của mỗi từ trong positive samples $freq_{pos}$ và trong negative samples $freq_{neg}$

- Tính xác suất likelihood mỗi từ trong bộ từ điển
    - Sử dụng hàm `lookup` lấy ra tần suất xuất hiện của từ là positive $freq_{pos}$, và tần xuất xuất hiện của từ là negative $freq_{neg}$
- Tính xác suất cho mỗi từ thuộc vào positive sample: $P(W_{pos})$, thuộc vào negative sample $P(W_{neg})$ sử dụng công thức 4 & 5.

$$ P(W_{pos}) = \frac{freq_{pos} + 1}{N_{pos} + V}\tag{4} $$
$$ P(W_{neg}) = \frac{freq_{neg} + 1}{N_{neg} + V}\tag{5} $$

**Note:** Chúng ta lưu trữ likelihood của mỗi từ vào dictionary với key (từ): $W$, value (dictionary): ${0: P(W_{pos}), 1: P(W_{pos})}$

In [8]:
# Ex 3
def compute_likelihood(freqs):
    # Tính xác suất likelihood của mỗi từ trong bộ từ điển

    ### START CODE HERE
    # Tính V các từ duy nhất xuất hiện trong tập train
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    # Tính N_pos: số lượng từ trong positive samples và N_neg: số từ trong negative sample
    N_pos = N_neg = 0
    for pair in freqs.keys():
        # Nếu như class: 1 tăng N_pos thêm số lần xuất hiện của pair trong freqs
        if pair[1] > 0:
            N_pos += freqs[pair]

        # Nếu như class: 0 tăng N_neg thêm số lần xuất hiện của pair trong freqs
        else:
            N_neg += freqs[pair]
    
    print(f'V: {V}, N_pos: {N_pos}, N_neg: {N_neg}')

    # Tính likelihood cho mỗi từ trong bộ từ điển
    p_likelihood = {}
    
    for word in tqdm.tqdm(vocab):
        # Lấy tần xuất xuất hiện của mỗi từ là positive hoặc negative
        freq_pos = lookup(freqs, word, 1)
        freq_neg = lookup(freqs, word, 0)

        # Tính xác suất likelihood của mỗi từ với class positive và negative
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)

        # Lưu vào p_likelihood dictionary
        p_likelihood[word] = {0:p_w_neg, 1:p_w_pos}
    # END CODE HERE
    
    return p_likelihood

In [9]:
# Kết quả mong đợi
compute_likelihood(freqs)

V: 20, N_pos: 9, N_neg: 14


100%|██████████| 20/20 [00:00<?, ?it/s]


{'energy': {0: 0.058823529411764705, 1: 0.034482758620689655},
 'film': {0: 0.029411764705882353, 1: 0.06896551724137931},
 'the': {0: 0.029411764705882353, 1: 0.10344827586206896},
 'no': {0: 0.058823529411764705, 1: 0.034482758620689655},
 'surprises': {0: 0.058823529411764705, 1: 0.034482758620689655},
 'of': {0: 0.029411764705882353, 1: 0.06896551724137931},
 'fun': {0: 0.029411764705882353, 1: 0.06896551724137931},
 'entirely': {0: 0.058823529411764705, 1: 0.034482758620689655},
 'lacks': {0: 0.058823529411764705, 1: 0.034482758620689655},
 'boring': {0: 0.058823529411764705, 1: 0.034482758620689655},
 'most': {0: 0.029411764705882353, 1: 0.06896551724137931},
 'very': {0: 0.058823529411764705, 1: 0.06896551724137931},
 'plain': {0: 0.058823529411764705, 1: 0.034482758620689655},
 'laughs': {0: 0.058823529411764705, 1: 0.034482758620689655},
 'summer': {0: 0.029411764705882353, 1: 0.06896551724137931},
 'and': {0: 0.08823529411764706, 1: 0.034482758620689655},
 'powerful': {0: 0.0

**Bước 3: Hoàn thiện `train` function cho Naive Bayes***

In [10]:
def train_naive_bayes(train_x, train_y):
    ''' Huấn luyện thuật toán Naive Bayes
    Args:
        train_x: Danh sách các câu
        train_y: Danh sách các nhãn tương ứng (0 hoặc 1)
    Output:
        p_prior: the prior probability (Xác suấ tiên nghiệm)
        p_likelihood: the maximum likelihood of the probability.
    '''
    # Xây dựng từ điển tần suất xuất hiện của từ và nhãn tương ứng
    freqs = count_freq_words(train_x, train_y)

    # Tính xác suất tiên nghiệm
    p_prior = compute_prior_prob(train_y)

    # Tính xác suất likelihood
    p_likelihood = compute_likelihood(freqs)

    return p_prior, p_likelihood

In [11]:
# Kết quả đầu ra thu được khi huấn luận Naive Bayes Classifier
p_prior, p_likelihood = train_naive_bayes(train_x, train_y)
p_prior, p_likelihood

5it [00:00, ?it/s]


V: 20, N_pos: 9, N_neg: 14


100%|██████████| 20/20 [00:00<?, ?it/s]


({0: 0.6, 1: 0.4},
 {'energy': {0: 0.058823529411764705, 1: 0.034482758620689655},
  'film': {0: 0.029411764705882353, 1: 0.06896551724137931},
  'the': {0: 0.029411764705882353, 1: 0.10344827586206896},
  'no': {0: 0.058823529411764705, 1: 0.034482758620689655},
  'surprises': {0: 0.058823529411764705, 1: 0.034482758620689655},
  'of': {0: 0.029411764705882353, 1: 0.06896551724137931},
  'fun': {0: 0.029411764705882353, 1: 0.06896551724137931},
  'entirely': {0: 0.058823529411764705, 1: 0.034482758620689655},
  'lacks': {0: 0.058823529411764705, 1: 0.034482758620689655},
  'boring': {0: 0.058823529411764705, 1: 0.034482758620689655},
  'most': {0: 0.029411764705882353, 1: 0.06896551724137931},
  'very': {0: 0.058823529411764705, 1: 0.06896551724137931},
  'plain': {0: 0.058823529411764705, 1: 0.034482758620689655},
  'laughs': {0: 0.058823529411764705, 1: 0.034482758620689655},
  'summer': {0: 0.029411764705882353, 1: 0.06896551724137931},
  'and': {0: 0.08823529411764706, 1: 0.034482

###1.4.Dự đoán với các mẫu thử nghiệm
- Tính xác suất của mỗi sample (n từ) dựa vào công thức:
$$P(0).P(S|0) = P(0).P(w_{1}|0).P(w_{2}|0)...P(w_{n}|0)$$ 

In [27]:
# Ex 4
import tqdm
def naive_bayes_predict(sentence, p_prior, p_likelihood):
    '''
    Args:
        sentence: a string
        p_prior: a dictionary of the prior probability
        p_likelihood: a dictionary of words mapping to the probability
    Output:
        p: the probability of sentence with 0: negative, 1: positive

    '''
    # Tiền xử lý dữ liệu
    words = basic_preprocess(sentence)

    # Khởi tạo giá trị xác suất ban đầu là giá trị xác suất tiên nghiệm
    p_neg = p_prior[0]
    p_pos = p_prior[1]
    # use tqdm to make a progress bar
    for word in words:
        # Kiểm tra xem word có tồn tại trong p_likelihood hay không
        if word in p_likelihood:
            ### START CODE HERE
            # nhân xác suất tiên nghiệm với xác suất likelihood của các từ
            p_neg *= p_likelihood[word][0]
            p_pos *= p_likelihood[word][1]
            ### END CODE HERE
    return {'prob': {0: p_neg, 1: p_pos},
            'label': 0 if p_neg > p_pos else 1}


In [13]:
# Kết quả mong đợi
sentence = "predictable with no fun"
naive_bayes_predict(sentence, p_prior, p_likelihood)

100%|██████████| 4/4 [00:00<?, ?it/s]


{'prob': {0: 6.106248727864848e-05, 1: 3.2801672885317154e-05}, 'label': 0}

##2.Naive Bayes Classfier for Sentiment Analysis on Tweets
**Phân tích cảm xúc trên tập 1Tweets1 sử dụng thuật toán phân loại Naive Bayes**

In [1]:
import re
import string
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
# read pandas
df_train = pd.read_csv(r'C:\Users\Asus\Documents\VINBIGDATA\Machine Learning\llm-detect-ai-generated-text\data_split\train.csv')
df_test = pd.read_csv(r'C:\Users\Asus\Documents\VINBIGDATA\Machine Learning\llm-detect-ai-generated-text\data_split\test.csv')

In [4]:
df_train

,id,prompt_id,text,generated
0,e_k7r6yo0s,8,"For the majority of students in the US, school...",1
1,e_dkh6chp9,2,Have you ever thought about what was the face ...,1
2,e_eenni7od,2,A face has been found by NASA's Viking 1 space...,1
3,e_pi969vyg,3,Do you know what the Seagoing Cowboys program ...,0
4,e_02r7rk2e,9,Students with a grade C average should be able...,0
...,...,...,...,...
126466,e_ldxdajal,5,The use of technology to read the emotional ex...,1
126467,e_f2zny2as,1,"Dear State Senator,\n\nI am writing to express...",1
126468,e_jhej2vio,1,"PROPER_NAME\n\nJanuary 21, 2015\n\nDear Senato...",1
126469,e_p7uvxihr,1,"Dear Senator,\n\nI am writing to you today to ...",1


In [6]:
df_test

,id,prompt_id,text,generated
0,e_8tt3v1l1,8,Summer projects should be student basedecausei...,1
1,e_yu4mige3,4,"My position on driverless cars is, that they w...",0
2,e_cmiw5i1g,5,I'm against to the use of this technology to r...,1
3,e_sdxf4v6a,8,Summer projects should be studentdesigned beca...,1
4,e_d4wsemmz,10,"Mom,"" Generic_Name whimpered sadly. ""Why is it...",1
...,...,...,...,...
15805,e_s148lfph,7,Technology has had a profound impact on people...,1
15806,e_lhpihz0f,2,I think the face on Mars is a landform because...,0
15807,e_r9mtyiw1,4,"2019, cars running on electricity and powered ...",1
15808,e_ri7bz18z,5,I feel as if the technology is too adavnced to...,1


###2.1.Dowload Dataset

In [5]:
import string
import re
import nltk
from nltk.corpus import stopwords
Stopwords = set(stopwords.words('english'))
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub('[^a-zA-Z0-9]',' ',text)
    text = re.sub('\s+',' ',text)
    
    new_text =[]
    for word in text.split():
        if word not in Stopwords:
            new_text.append(word)
    
    ps = nltk.stem.porter.PorterStemmer()
    new_text = [ps.stem(word) for word in new_text]
    #lem = nltk.stem.wordnet.WordNetLemmatizer()
    #new_text = [lem.lemmatize(word) for word in new_text]
    
    text =" ".join(new_text)
    return(text)

In [7]:
# extract text and label
train_x = df_train['text'].apply(preprocess_text).values
train_y = df_train['generated']

test_x = df_test['text'].apply(preprocess_text).values
test_y = df_test['generated']

###2.3.Huấn luyện Naive Bayes Classifier trên tập `Tweets`

In [23]:
import tqdm

In [24]:
p_prior, p_likelihood = train_naive_bayes(train_x, train_y)

126471it [00:20, 6305.50it/s]


V: 508504, N_pos: 38198059, N_neg: 12020923


100%|██████████| 508504/508504 [00:00<00:00, 550875.95it/s]


In [30]:
#Kết quả ví dụ về xác suất tiên nghiệm và likelihood của từ happy
p_prior, p_likelihood['happy']

({0: 0.228518790869053, 1: 0.771481209130947},
 {0: 0.00019961008592013027, 1: 0.00022910843310990954})

###2.4.Dự đoán

In [29]:
test_x[0], test_y[0]

("Summer projects should be student basedecauseit will encourage them to do a topic of their liking. Students should be able to choose their own summer projectsbecausethey will be more creative and enjoy it more. Students should be able to pick what they want to do for their summer projectsecauseit will lead to more creativity. Students will also be able to incorporate their summer experiences into their projects. Although, students may not want to do it over summer, they should be required to complete a summer project. Although, if they were to make their project about their summer break they would be able to tell a story and show what they learned. Although students don't want to do summer projects, they will if they can pick what they do.",
 np.int64(1))

In [28]:
naive_bayes_predict(test_x[0], p_prior, p_likelihood)

{'prob': {0: 0.0, 1: 0.0}, 'label': 1}

###2.5.Đánh giá độ chính xác trên tập test

In [31]:
# acc = 0
# for sentence, label in zip(test_x, test_y):

#     # predic each sentence in test set
#     pred = naive_bayes_predict(sentence, p_prior, p_likelihood)['label']

#     # compare predict label with target label
#     if int(pred) == int(label):
#         acc += 1

# # calculate prediction, recall, f1 score and show confusion matrix

#     # 

# print('Accuracy: ', acc/len(test_x))

Accuracy:  0.763124604680582


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'C:\Users\Asus\Documents\VINBIGDATA\Machine Learning\llm-detect-ai-generated-text\mistral7b_all_clean.csv')

df

,id,prompt_id,text,generated,mistral7b
0,e_7iqq3rsd,2,recent year mani citicen state call facelik fa...,1,1
1,e_kl8n58ii,1,elector colleg countri way give presid peopl t...,1,1
2,e_z6e91t82,6,articl author believ need tri srvey venu despi...,1,1
3,e_y6pkid7o,1,dear state senat write express opinion topic e...,1,0
4,e_uh78br5j,0,mani us use car compuls sometim without think ...,1,0
...,...,...,...,...,...
15805,e_23mslsms,12,ever friend teacher person give advic know muc...,1,1
15806,e_919wpoz9,2,face mar ever seen landmark look like someth e...,0,0
15807,e_vqgunhtn,6,articl author tell us fact venu call even star...,0,1
15808,e_orzvfeey,2,face mar martian mesa locat cydonia subject mu...,1,0


In [33]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
print("Model Naive Bayes")
# Initialize variables
acc = 0
y_true = []
y_pred = []

for sentence, label in zip(test_x, test_y):

    # Predict each sentence in the test set
    pred = naive_bayes_predict(sentence, p_prior, p_likelihood)['label']

    # Store the true labels and predicted labels
    y_true.append(int(label))
    y_pred.append(int(pred))

    # Compare predict label with target label
    if int(pred) == int(label):
        acc += 1

# Calculate accuracy
accuracy = acc / len(test_x)
print('Accuracy: ', accuracy)

# Calculate and print confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:\n', conf_matrix)

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print('Precision: ', precision)
print('Recall: ', recall)
print('F1 Score: ', f1)


Model Naive Bayes
Accuracy:  0.763124604680582
Confusion Matrix:
 [[    2  3658]
 [   87 12063]]
Precision:  0.5948865299841173
Recall:  0.763124604680582
F1 Score:  0.6654851988241132


In [ ]:
# use 

##3.Logistic Regression for Sentiment Analysis on Tweets

In [9]:
import numpy as np

In [10]:
# use library sklearn to compare with our model
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

model = make_pipeline(CountVectorizer(), MultinomialNB())
model.fit(train_x, train_y)
pred = model.predict(test_x)

# Calculate accuracy
accuracy = np.mean(pred == test_y)
print('Accuracy: ', accuracy)
# show confusion matrix
conf_matrix = confusion_matrix(test_y, pred)
print('Confusion Matrix:\n', conf_matrix)

# Calculate precision, recall, and F1 score

precision = precision_score(test_y, pred, average='weighted')
recall = recall_score(test_y, pred, average='weighted')
f1 = f1_score(test_y, pred, average='weighted')

print('Precision: ', precision)
print('Recall: ', recall)
print('F1 Score: ', f1)




Accuracy:  0.6659076533839342
Confusion Matrix:
 [[2657 1003]
 [4279 7871]]
Precision:  0.7703209557837707
Recall:  0.6659076533839342
F1 Score:  0.6915244104291606


In [7]:
!pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 330.3 kB/s eta 0:06:19
   ---------------------------------------- 0.0/124.9 MB 393.8 kB/s eta 0:05:18
   ---------------------------------------- 0.1/124.9 MB 409.6 kB/s eta 0:05:05
   ---------------------------------------- 0.1/124.9 MB 722.1 kB/s eta 0:02:53
   ---------------------------------------- 0.2/124.9 MB 1.1 MB/s eta 0:01:53
   ---------------------------------------- 0.4/124.9 MB 1.5 MB/s eta 0:01:23
   ---------------------------------------- 0.5/124.9 MB 1.8 MB/s eta 0:01:09
   ---------------------------------------- 0.9/124.9 MB 2.7 MB/s eta 0:00:47
   ---------------------------------------- 1.2/124.9 MB 3.4 MB/s eta 0:00:37
    --------------------------------------- 1.6/124.9 MB 3.9 MB/s eta 0:00:32
    --------------------------------------- 2.1/124.9 MB 4.7 MB/s eta 

In [11]:
# use model XGBClassifier to compare with our model
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

model = make_pipeline(CountVectorizer(), XGBClassifier())
model.fit(train_x, train_y)
pred = model.predict(test_x)
    
# Calculate accuracy
accuracy = np.mean(pred == test_y)
print('Accuracy: ', accuracy)
# show confusion matrix
conf_matrix = confusion_matrix(test_y, pred)
print('Confusion Matrix:\n', conf_matrix)

# Calculate precision, recall, and F1 score

precision = precision_score(test_y, pred, average='weighted')
recall = recall_score(test_y, pred, average='weighted')
f1 = f1_score(test_y, pred, average='weighted')

print('Precision: ', precision)
print('Recall: ', recall)
print('F1 Score: ', f1)


Accuracy:  0.8444022770398482
Confusion Matrix:
 [[ 1633  2027]
 [  433 11717]]
Precision:  0.8381410935500208
Recall:  0.8444022770398482
F1 Score:  0.8275338765347617


In [12]:
# use model SGD Classifier to compare with our model
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

model = make_pipeline(CountVectorizer(), SGDClassifier())
model.fit(train_x, train_y)
pred = model.predict(test_x)

# Calculate accuracy
accuracy = np.mean(pred == test_y)
print('Accuracy: ', accuracy)
# show confusion matrix
conf_matrix = confusion_matrix(test_y, pred)
print('Confusion Matrix:\n', conf_matrix)

# Calculate precision, recall, and F1 score

precision = precision_score(test_y, pred, average='weighted')
recall = recall_score(test_y, pred, average='weighted')
f1 = f1_score(test_y, pred, average='weighted')

print('Precision: ', precision)
print('Recall: ', recall)
print('F1 Score: ', f1)


Accuracy:  0.8533839342188488
Confusion Matrix:
 [[ 2027  1633]
 [  685 11465]]
Precision:  0.8457144524434377
Recall:  0.8533839342188488
F1 Score:  0.8452300015041799


In [13]:
# use library sklearn use logistic regression to compare with our model
from sklearn.linear_model import LogisticRegression
model = make_pipeline(CountVectorizer(), LogisticRegression())
model.fit(train_x, train_y)
pred = model.predict(test_x)

# Calculate accuracy
accuracy = np.mean(pred == test_y)
print('Accuracy: ', accuracy)
# show confusion matrix
conf_matrix = confusion_matrix(test_y, pred)
print('Confusion Matrix:\n', conf_matrix)
# Calculate precision, recall, and F1 score

precision = precision_score(test_y, pred, average='weighted')
recall = recall_score(test_y, pred, average='weighted')
f1 = f1_score(test_y, pred, average='weighted')

print('Precision: ', precision)
print('Recall: ', recall)
print('F1 Score: ', f1)

c:\Users\Asus\anaconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy:  0.8368753953194181
Confusion Matrix:
 [[ 1816  1844]
 [  735 11415]]
Precision:  0.8264204251383845
Recall:  0.8368753953194181
F1 Score:  0.8258719698144881
